In [1]:
import os
import re
from bs4 import BeautifulSoup
from collections import defaultdict
import pickle

In [2]:
data_dir = './data_html/'

In [3]:
# List all html files from data directory 
# with file name length 15 (meaning particular date like: '2023-01-01.html')

# poznámka: 
# keďže cirkevný rok začína adventom, sú v zip archíve aj dni z predošlého roka (2022)
# preto som pre názornosť dal podmienku, aby bral len aktuálny rok 2023 (file.startswith('2023'))
html_files = [ file for file in os.listdir(data_dir) 
                    if file.startswith('2023') and file.endswith(".html") and len(file)==15 ]

len(html_files) 

365

In [4]:
mydict = defaultdict(dict)

for fname in html_files:

    datum = fname.removesuffix('.html')

    with open(data_dir+fname, encoding='utf-8') as f:
        s = f.read()    
        
    soup = BeautifulSoup(s,'lxml') #     

    # meniny:
    meniny = soup.find('span',class_='lcND').get_text()
    meniny = meniny.removeprefix('meniny: ')
    mydict[datum].update({'meniny':meniny})

    elems = soup.find_all('div', class_='lcDENalt') # tu riešim vigíliu.
    for elem in elems:
        h3 = elem.find('h3')                       
        if h3:
            if h3.get_text().startswith('Vigília'): # ak je to vigília, mažeme.
                elem.extract()  

    # kruciálny odsek:
    elems = soup.find_all('h2')
    for i, elem in enumerate(elems):
        ciel = []
        ciel.append(elem.get_text('\n',True))
        toc_title = elem['title']
        vysledny_string = '\n'.join(ciel)
        vysledny_string = re.sub(r'\n(\[\w+\])', r' \1', vysledny_string) # replacing '\n[Biela]' with ' [Biela]...
        # vysledny_string = re.sub(r'\n(\(.+\))', r' \1', vysledny_string) # replacing "\n(ľubovoľná spomienka)" with " (ľubovoľná spomienka)"
        mydict[datum].update({i:[vysledny_string,toc_title]})

c:\Python\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [5]:
print(mydict['2023-01-03']) # test zobrazenia

{'meniny': 'Daniela', 0: ['Utorok vo Vianočnom období [Biela]\nalebo\nNajsvätejšieho mena Ježiš [Biela]\n(ľubovoľná spomienka)', 'Vianočné obdobie; lit. rok: 2023; ned./fer. cyklus: A/I']}


In [6]:
print(mydict['2023-07-17']) # test zobrazenia

{'meniny': 'Bohuslav', 0: ['Svätých Andreja-Svorada a Benedikta, pustovníkov [Biela]\n(spomienka)', 'Cezročné obdobie; lit. rok: 2023; ned./fer. cyklus: A/I'], 1: ['NITRA: Svätých Andreja-Svorada a Benedikta, pustovníkov – hlavných patrónov diecézy [Biela]\n(sviatok)', 'Cezročné obdobie; lit. rok: 2023; ned./fer. cyklus: A/I'], 2: ['Mesto Nitra: Svätého Andreja-Svorada – patróna mesta [Biela]\n(slávnosť)', 'Cezročné obdobie; lit. rok: 2023; ned./fer. cyklus: A/I']}


In [7]:
# serializovať do pickle.
# Serialize and save the dictionary to a pickle file
with open('mydict_dict.pickle', 'wb') as file:
    pickle.dump(mydict, file)

In [8]:
# Load the dictionary from the pickle file
with open('mydict_dict.pickle', 'rb') as file:
    loaded_dict = pickle.load(file)

In [9]:
mydict == loaded_dict

True

In [11]:
# done